<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TFDS/TFRecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

ds = tf.data.Dataset.range(100)
ds = ds.window(size=11, shift=1)
ds = ds.flat_map(lambda w: w.batch(
    batch_size=11, drop_remainder=True))
ds = ds.map(lambda w: (w[:-1], w[-1]))

next(iter(ds))

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=10>)

In [2]:
@tf.py_function(Tout=tf.string)
def serialize(x, y):
        x = tf.io.serialize_tensor(x).numpy()
        x_feature = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[x]))

        y = tf.io.serialize_tensor(y).numpy()
        y_feature = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[y]))

        proto = tf.train.Example(features=tf.train.Features(feature={
            "x": x_feature,
            "y": y_feature
        }))
        return proto.SerializeToString()

x, y = next(iter(ds))
serialized_sample = serialize(x, y)
proto = tf.train.Example.FromString(serialized_sample.numpy())
proto

features {
  feature {
    key: "x"
    value {
      bytes_list {
        value: "\010\t\022\004\022\002\010\n\"P\000\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\002\000\000\000\000\000\000\000\003\000\000\000\000\000\000\000\004\000\000\000\000\000\000\000\005\000\000\000\000\000\000\000\006\000\000\000\000\000\000\000\007\000\000\000\000\000\000\000\010\000\000\000\000\000\000\000\t\000\000\000\000\000\000\000"
      }
    }
  }
  feature {
    key: "y"
    value {
      bytes_list {
        value: "\010\t\022\000\"\010\n\000\000\000\000\000\000\000"
      }
    }
  }
}

In [4]:
with tf.io.TFRecordWriter("test.tfrecord") as writer:
    for x, y in ds:
        sample = serialize(x, y)
        writer.write(sample.numpy())

In [13]:
features = {
    "x": tf.io.FixedLenFeature(shape=(10,), dtype=tf.float32),
    "y": tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
}
def _parse_function(serialized):
    return tf.io.parse_single_example(serialized, features=features)

ds = tf.data.TFRecordDataset("test.tfrecord")
parsed_ds = ds.map(_parse_function)

for r in parsed_ds.take(10):
    print(r)

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: y.  Data types don't match. Data type: string but expected type: float
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 